Open university DB

In [7]:
from pyspark.sql import SparkSession

if 'spark' in locals():
    spark.stop()
    print("Spark session stopped")

# Initialize Spark session
spark = SparkSession.builder \
    .appName("OULAD Analysis") \
    .getOrCreate()

# Define the file paths
assessments_path = "assessments.csv"
courses_path = "courses.csv"
student_assessment_path = "studentAssessment.csv"
student_info_path = "studentInfo.csv"
student_registration_path = "studentRegistration.csv"
student_vle_path = "studentVle.csv"
vle_path = "vle.csv"

# Load the dataset into DataFrames
assessments_df = spark.read.option("header", "true").csv(assessments_path)
courses_df = spark.read.option("header", "true").csv(courses_path)
student_assessment_df = spark.read.option("header", "true").csv(student_assessment_path)
student_info_df = spark.read.option("header", "true").csv(student_info_path)
student_registration_df = spark.read.option("header", "true").csv(student_registration_path)
student_vle_df = spark.read.option("header", "true").csv(student_vle_path)
vle_df = spark.read.option("header", "true").csv(vle_path)

Spark session stopped


In [11]:
print("Courses DataFrame:")
courses_df.show(5)

print("Number of rows in Assessments DataFrame:", assessments_df.count())

student_info_df.select("id_student", "gender", "highest_education").show(5)



Courses DataFrame:
+-----------+-----------------+--------------------------+
|code_module|code_presentation|module_presentation_length|
+-----------+-----------------+--------------------------+
|        AAA|            2013J|                       268|
|        AAA|            2014J|                       269|
|        BBB|            2013J|                       268|
|        BBB|            2014J|                       262|
|        BBB|            2013B|                       240|
+-----------+-----------------+--------------------------+
only showing top 5 rows

Number of rows in Assessments DataFrame: 206
+----------+------+--------------------+
|id_student|gender|   highest_education|
+----------+------+--------------------+
|     11391|     M|    HE Qualification|
|     28400|     F|    HE Qualification|
|     30268|     F|A Level or Equiva...|
|     31604|     F|A Level or Equiva...|
|     32885|     F|  Lower Than A Level|
+----------+------+--------------------+
only showin

Filter rows

In [13]:
female_students = student_info_df.filter(student_info_df.gender == "F")
female_students.show(5)

+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|code_module|code_presentation|id_student|gender|              region|   highest_education|imd_band|age_band|num_of_prev_attempts|studied_credits|disability|final_result|
+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|        AAA|            2013J|     28400|     F|            Scotland|    HE Qualification|  20-30%|   35-55|                   0|             60|         N|        Pass|
|        AAA|            2013J|     30268|     F|North Western Region|A Level or Equiva...|  30-40%|   35-55|                   0|             60|         Y|   Withdrawn|
|        AAA|            2013J|     31604|     F|   South East Region|A Level or Equiva...|  50-60%|   35-55|                   0|             60

Group operations

In [18]:
from pyspark.sql.functions import count, avg
gender_count_df = student_info_df.groupby("gender").agg(count("id_student").alias("count"))

gender_count_df.show()

# Count the number of students per course
students_per_course_df = student_registration_df.groupBy("code_presentation").agg(count("id_student").alias("total_students"))
students_per_course_df.show()

# Calculate the average score for each assessment
avg_score_per_assessment_df = student_assessment_df.groupBy("id_assessment").agg(avg("score").alias("avg_score"))
avg_score_per_assessment_df.show()


+------+-----+
|gender|count|
+------+-----+
|     F|14718|
|     M|17875|
+------+-----+

+-----------------+--------------+
|code_presentation|total_students|
+-----------------+--------------+
|            2013J|          8845|
|            2014J|         11260|
|            2014B|          7804|
|            2013B|          4684|
+-----------------+--------------+

+-------------+-----------------+
|id_assessment|        avg_score|
+-------------+-----------------+
|        25349|73.92911585365853|
|        25350|77.25956061838893|
|        24285|69.29948364888124|
|        30710|82.69076305220884|
|        34874|72.45355850422196|
|        14986|70.97363465160075|
|        34868|78.36876355748373|
|        14991| 93.8099243061396|
|        15006|86.42434488588334|
|        14984|71.21529324424647|
|        25353|60.20640569395018|
|        25364|75.01545972335232|
|        34879| 87.9689349112426|
|        34871|76.39145106861642|
|        25341|66.61584158415842|
|        30722|7

Joins

In [19]:
joined_df = student_info_df.join(student_assessment_df, "id_student")
joined_df.select("id_student", "gender", "score").show(5)

+----------+------+-----+
|id_student|gender|score|
+----------+------+-----+
|     11391|     M|   78|
|     28400|     F|   70|
|     31604|     F|   72|
|     32885|     F|   69|
|     38053|     M|   79|
+----------+------+-----+
only showing top 5 rows



In [20]:
spark.stop()